# PART 2 - Transformations & Jointures avec PySpark
## Projet E-Commerce - Soraya PITROIPA

---

**Objectifs** :
- Effectuer des jointures entre les datasets Customers et Orders
- Réaliser des agrégations complexes (CA par client, catégorie, pays)
- Utiliser les fonctions de fenêtrage (Window Functions)
- Calculer des métriques métier (RFM, clients actifs, taux de rétention)
- Créer des visualisations des résultats

---

## 1. Configuration Spark & Chargement des Données

In [1]:
# Configuration des variables d'environnement pour Spark sur Windows
import os
os.environ['JAVA_HOME'] = r"C:\Program Files\Microsoft\jdk-17.0.17.10-hotspot"
os.environ['HADOOP_HOME'] = r"C:\hadoop"

print(" Variables d'environnement configurées")
print(f"   JAVA_HOME: {os.environ['JAVA_HOME']}")
print(f"   HADOOP_HOME: {os.environ['HADOOP_HOME']}")

 Variables d'environnement configurées
   JAVA_HOME: C:\Program Files\Microsoft\jdk-17.0.17.10-hotspot
   HADOOP_HOME: C:\hadoop


In [2]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import (
    col, count, sum as spark_sum, avg, max as spark_max, min as spark_min,
    datediff, current_date, when, round as spark_round,
    row_number, rank, dense_rank, lag, lead,
    year, month, dayofmonth, to_date
)
from pyspark.sql.window import Window
import matplotlib.pyplot as plt
import seaborn as sns

# Configuration
spark = SparkSession.builder \
    .appName("Part2_Transformations_Jointures") \
    .config("spark.driver.memory", "4g") \
    .getOrCreate()

print(f"Spark version: {spark.version}")

Spark version: 4.0.1


In [3]:
# Chargement des données nettoyées (Part 1)
import os

# Vérifier les chemins possibles
paths_to_try = [
	("../data/processed/customers_clean.parquet", "../data/processed/orders_clean.parquet"),
	("./data/processed/customers_clean.parquet", "./data/processed/orders_clean.parquet"),
	("data/processed/customers_clean.parquet", "data/processed/orders_clean.parquet"),
]

df_customers = None
df_orders = None

for customers_path, orders_path in paths_to_try:
	if os.path.exists(customers_path) and os.path.exists(orders_path):
		print(f"✓ Fichiers trouvés dans: {customers_path}")
		df_customers = spark.read.parquet(customers_path)
		df_orders = spark.read.parquet(orders_path)
		break

if df_customers is None or df_orders is None:
	print("❌ Fichiers parquet non trouvés. Veuillez vérifier que Part 1 a été exécutée.")
	print(f"Répertoire actuel: {os.getcwd()}")
	print("\nVeuillez exécuter le notebook Part 1 d'abord pour générer les fichiers nettoyés.")
else:
	print(f"Customers: {df_customers.count():,} lignes")
	print(f"Orders: {df_orders.count():,} lignes")

✓ Fichiers trouvés dans: ./data/processed/customers_clean.parquet


Py4JJavaError: An error occurred while calling o30.parquet.
: java.lang.UnsatisfiedLinkError: 'boolean org.apache.hadoop.io.nativeio.NativeIO$Windows.access0(java.lang.String, int)'
	at org.apache.hadoop.io.nativeio.NativeIO$Windows.access0(Native Method)
	at org.apache.hadoop.io.nativeio.NativeIO$Windows.access(NativeIO.java:817)
	at org.apache.hadoop.fs.FileUtil.canRead(FileUtil.java:1415)
	at org.apache.hadoop.fs.FileUtil.list(FileUtil.java:1620)
	at org.apache.hadoop.fs.RawLocalFileSystem.listStatus(RawLocalFileSystem.java:739)
	at org.apache.hadoop.fs.FileSystem.listStatus(FileSystem.java:2078)
	at org.apache.hadoop.fs.FileSystem.listStatus(FileSystem.java:2122)
	at org.apache.hadoop.fs.ChecksumFileSystem.listStatus(ChecksumFileSystem.java:961)
	at org.apache.spark.util.HadoopFSUtils$.listLeafFiles(HadoopFSUtils.scala:218)
	at org.apache.spark.util.HadoopFSUtils$.$anonfun$parallelListLeafFilesInternal$1(HadoopFSUtils.scala:132)
	at scala.collection.immutable.List.map(Li

In [ ]:
# Vérification des schémas
print("=== Schema Customers ===")
df_customers.printSchema()

print("\n=== Schema Orders ===")
df_orders.printSchema()

=== Schema Customers ===


AttributeError: 'NoneType' object has no attribute 'printSchema'

## 2. Jointure Customers ⋈ Orders

###  Justification de la Jointure

**Type de jointure : INNER JOIN**

**Raison du choix :**
- Nous voulons analyser uniquement les commandes associées à des clients valides
- Élimine les commandes orphelines (customer_id inexistant dans customers)
- Garantit l'intégrité référentielle des analyses

**Cardinalité : 1-N (One-to-Many)**
- **1 Client** peut avoir **N Commandes** (0, 1 ou plusieurs)
- Relation classique dans un système e-commerce
- La clé de jointure `customer_id` est :
  - **Clé primaire** dans `customers` (unicité garantie)
  - **Clé étrangère** dans `orders` (répétitions autorisées)

**Impact attendu :**
- Perte de commandes sans client valide (données incohérentes)
- Conservation de tous les clients actifs ayant passé au moins 1 commande

In [ ]:
# Jointure INNER (garder uniquement les commandes avec client valide)
df_joined = df_orders.join(
    df_customers,
    on="customer_id",
    how="inner"
)

print(f"Lignes après jointure: {df_joined.count():,}")
print(f"Colonnes: {len(df_joined.columns)}")

# Aperçu
df_joined.select(
    "order_id", "customer_id", "first_name", "last_name",
    "order_date", "product_category", "total_amount", "customer_segment"
).show(10, truncate=False)

AttributeError: 'NoneType' object has no attribute 'join'

In [ ]:
# Vérification de la qualité de la jointure
print("=== Vérification Jointure ===")
print(f"Orders avant jointure: {df_orders.count():,}")
print(f"Orders après jointure: {df_joined.count():,}")
print(f"Perte: {((df_orders.count() - df_joined.count()) / df_orders.count() * 100):.2f}%")

=== Vérification Jointure ===


AttributeError: 'NoneType' object has no attribute 'count'

## 3. Agrégations Principales

### 3.1 Chiffre d'Affaires par Client

In [ ]:
# CA par client avec statistiques
df_ca_client = df_joined.groupBy("customer_id", "first_name", "last_name", "customer_segment").agg(
    count("order_id").alias("nb_commandes"),
    spark_sum("total_amount").alias("ca_total"),
    avg("total_amount").alias("ca_moyen"),
    spark_max("total_amount").alias("ca_max"),
    spark_min("total_amount").alias("ca_min")
).orderBy(col("ca_total").desc())

# Arrondir les valeurs
df_ca_client = df_ca_client.withColumn("ca_total", spark_round("ca_total", 2)) \
    .withColumn("ca_moyen", spark_round("ca_moyen", 2)) \
    .withColumn("ca_max", spark_round("ca_max", 2)) \
    .withColumn("ca_min", spark_round("ca_min", 2))

print("=== TOP 10 Clients par CA ===")
df_ca_client.show(10, truncate=False)

In [ ]:
# Statistiques globales
print("=== Statistiques CA Global ===")
df_ca_client.select(
    spark_sum("ca_total").alias("ca_total_global"),
    avg("ca_total").alias("ca_moyen_par_client"),
    avg("nb_commandes").alias("nb_commandes_moyen")
).show()

### 3.2 Chiffre d'Affaires par Pays

In [ ]:
# CA par pays
df_ca_pays = df_joined.groupBy("country").agg(
    count("order_id").alias("nb_commandes"),
    spark_sum("total_amount").alias("ca_total"),
    avg("total_amount").alias("ca_moyen"),
    count("customer_id").alias("nb_clients_actifs")
).orderBy(col("ca_total").desc())

df_ca_pays = df_ca_pays.withColumn("ca_total", spark_round("ca_total", 2)) \
    .withColumn("ca_moyen", spark_round("ca_moyen", 2))

print("=== TOP 10 Pays par CA ===")
df_ca_pays.show(10, truncate=False)

### 3.3 Chiffre d'Affaires par Catégorie de Produit

In [ ]:
# CA par catégorie
df_ca_categorie = df_joined.groupBy("product_category").agg(
    count("order_id").alias("nb_commandes"),
    spark_sum("total_amount").alias("ca_total"),
    avg("total_amount").alias("ca_moyen"),
    spark_sum("quantity").alias("quantite_totale")
).orderBy(col("ca_total").desc())

df_ca_categorie = df_ca_categorie.withColumn("ca_total", spark_round("ca_total", 2)) \
    .withColumn("ca_moyen", spark_round("ca_moyen", 2))

print("=== CA par Catégorie de Produit ===")
df_ca_categorie.show(truncate=False)

### 3.4 Chiffre d'Affaires par Segment Client

In [ ]:
# CA par segment client
df_ca_segment = df_joined.groupBy("customer_segment").agg(
    count("customer_id").alias("nb_clients"),
    count("order_id").alias("nb_commandes"),
    spark_sum("total_amount").alias("ca_total"),
    avg("total_amount").alias("ca_moyen_commande")
).orderBy(col("ca_total").desc())

df_ca_segment = df_ca_segment.withColumn("ca_total", spark_round("ca_total", 2)) \
    .withColumn("ca_moyen_commande", spark_round("ca_moyen_commande", 2))

print("=== CA par Segment Client ===")
df_ca_segment.show(truncate=False)

In [ ]:
# Panier moyen par client
df_panier_moyen = df_joined.groupBy("customer_id", "first_name", "last_name", "customer_segment").agg(
    count("order_id").alias("nb_commandes"),
    spark_sum("total_amount").alias("ca_total"),
    avg("total_amount").alias("panier_moyen")
).withColumn("panier_moyen", spark_round("panier_moyen", 2)) \
 .withColumn("ca_total", spark_round("ca_total", 2)) \
 .orderBy(col("panier_moyen").desc())

print("=== TOP 10 Clients par Panier Moyen ===")
df_panier_moyen.show(10, truncate=False)

# Panier moyen global
panier_moyen_global = df_joined.agg(avg("total_amount").alias("panier_moyen_global")).collect()[0][0]
print(f"\n Panier Moyen Global : {panier_moyen_global:.2f} €")

# Panier moyen par segment
df_panier_segment = df_joined.groupBy("customer_segment").agg(
    count("order_id").alias("nb_commandes"),
    avg("total_amount").alias("panier_moyen")
).withColumn("panier_moyen", spark_round("panier_moyen", 2)) \
 .orderBy(col("panier_moyen").desc())

print("\n=== Panier Moyen par Segment ===")
df_panier_segment.show(truncate=False)

### 3.5 Panier Moyen

Le **panier moyen** représente le montant moyen dépensé par commande. C'est un indicateur clé de performance (KPI) pour mesurer la valeur moyenne des transactions.

## 4. Window Functions - Analyses Temporelles

In [ ]:
# Calcul RFM par client
from pyspark.sql.functions import ntile

# 1. Calculer les métriques RFM pour chaque client
df_rfm_base = df_joined.groupBy("customer_id", "first_name", "last_name", "email", "customer_segment").agg(
    datediff(current_date(), spark_max("order_date")).alias("recency"),  # Jours depuis dernier achat
    count("order_id").alias("frequency"),  # Nombre de commandes
    spark_sum("total_amount").alias("monetary")  # CA total
).withColumn("monetary", spark_round("monetary", 2))

print("=== Métriques RFM Brutes ===")
df_rfm_base.orderBy(col("monetary").desc()).show(10, truncate=False)

# 2. Calculer les scores RFM (1-5) avec ntile
# Note: Pour Recency, plus le score est élevé, plus c'est récent (inverse de la valeur)
window_spec = Window.orderBy(col("recency"))
window_spec_desc = Window.orderBy(col("frequency").desc())
window_spec_desc2 = Window.orderBy(col("monetary").desc())

df_rfm_scores = df_rfm_base.withColumn(
    "R_score", 
    6 - ntile(5).over(window_spec)  # Inverser: récent = score élevé
).withColumn(
    "F_score",
    ntile(5).over(window_spec_desc)
).withColumn(
    "M_score",
    ntile(5).over(window_spec_desc2)
)

# 3. Créer le score RFM combiné
df_rfm_final = df_rfm_scores.withColumn(
    "RFM_score",
    col("R_score") * 100 + col("F_score") * 10 + col("M_score")
).withColumn(
    "RFM_segment",
    when(col("RFM_score") >= 444, "Champions")  # 444-555
    .when(col("RFM_score") >= 344, "Loyal Customers")  # 344-443
    .when(col("RFM_score") >= 244, "Potential Loyalists")  # 244-343
    .when(col("RFM_score") >= 144, "At Risk")  # 144-243
    .otherwise("Lost")  # < 144
).orderBy(col("RFM_score").desc())

print("\n=== TOP 20 Clients - Analyse RFM ===")
df_rfm_final.select(
    "customer_id", "first_name", "last_name", 
    "recency", "frequency", "monetary",
    "R_score", "F_score", "M_score", "RFM_score", "RFM_segment"
).show(20, truncate=False)

# 4. Distribution par segment RFM
df_rfm_distribution = df_rfm_final.groupBy("RFM_segment").agg(
    count("customer_id").alias("nb_clients"),
    avg("monetary").alias("ca_moyen"),
    spark_sum("monetary").alias("ca_total")
).withColumn("ca_moyen", spark_round("ca_moyen", 2)) \
 .withColumn("ca_total", spark_round("ca_total", 2)) \
 .orderBy(col("ca_total").desc())

print("\n=== Distribution par Segment RFM ===")
df_rfm_distribution.show(truncate=False)

## 3.6 Analyse RFM (Recency, Frequency, Monetary)

**RFM** est une méthode de segmentation client basée sur 3 dimensions :
- **R - Recency (Récence)** : Temps écoulé depuis le dernier achat (plus récent = meilleur)
- **F - Frequency (Fréquence)** : Nombre de commandes (plus élevé = meilleur)
- **M - Monetary (Montant)** : CA total généré (plus élevé = meilleur)

Chaque dimension est notée de 1 à 5 (5 = meilleur), permettant d'identifier les meilleurs clients.

In [ ]:
# 3. Créer un dataset sécurisé pour export/partage
df_customers_export = df_customers_masked_v2.select(
    col("customer_id_hash").alias("customer_id_anonyme"),
    col("email_hash").alias("email_anonyme"),
    col("first_name"),
    col("last_name"),
    col("phone_masked_v2").alias("phone"),
    col("city"),
    col("state"),
    col("country"),
    col("customer_segment"),
    col("total_spent"),
    col("registration_date")
)

print("\n=== Dataset Clients Sécurisé (Prêt pour Export) ===")
df_customers_export.show(10, truncate=False)

# Statistiques de sécurisation
total_clients = df_customers.count()
print(f"\n Sécurisation Complétée :")
print(f"   - {total_clients:,} clients sécurisés")
print(f"   - customer_id → Hash SHA-256 (64 caractères)")
print(f"   - email → Hash SHA-256 (64 caractères)")
print(f"   - phone → Masqué (XXX-XXX-{'{derniers 4 chiffres}'})")
print(f"   - Conformité RGPD : Anonymisation des données personnelles")

In [ ]:
# 2. Masquage du numéro de téléphone
# Format original : +1-234-5674-8900 ou similaire
# Format masqué : XXX-XXX-X900 (garder les 4 derniers chiffres)

def mask_phone_udf(phone):
    """Masque un numéro de téléphone en gardant les 4 derniers chiffres"""
    if phone is None or phone == "Unknown":
        return "XXX-XXX-XXXX"
    
    # Extraire uniquement les chiffres
    digits_only = ''.join(filter(str.isdigit, str(phone)))
    
    if len(digits_only) >= 4:
        # Garder les 4 derniers chiffres
        last_4 = digits_only[-4:]
        masked = f"XXX-XXX-{last_4}"
    else:
        masked = "XXX-XXX-XXXX"
    
    return masked

# Enregistrer l'UDF
from pyspark.sql.types import StringType
mask_phone = spark.udf.register("mask_phone", mask_phone_udf, StringType())

# Appliquer le masquage
df_customers_masked = df_customers_secure.withColumn(
    "phone_masked",
    mask_phone(col("phone"))
)

print("\n=== Masquage des Numéros de Téléphone ===")
df_customers_masked.select(
    "customer_id", "first_name", "last_name",
    "phone", "phone_masked"
).show(10, truncate=False)

# Alternative avec regex (plus performant pour Spark)
df_customers_masked_v2 = df_customers_secure.withColumn(
    "phone_masked_v2",
    when(
        (col("phone").isNull()) | (col("phone") == "Unknown"),
        lit("XXX-XXX-XXXX")
    ).otherwise(
        concat_ws(
            "-",
            lit("XXX"),
            lit("XXX"),
            regexp_replace(col("phone"), r"^.*(\d{4})$", "$1")
        )
    )
)

print("\n=== Masquage avec Regex (Version Optimisée) ===")
df_customers_masked_v2.select(
    "customer_id", "phone", "phone_masked_v2"
).show(10, truncate=False)

In [ ]:
# Import des fonctions de sécurité
from pyspark.sql.functions import sha2, concat_ws, substring, length, regexp_replace, lit

# 1. Hash SHA-256 pour customer_id et email
df_customers_secure = df_customers.withColumn(
    "customer_id_hash",
    sha2(col("customer_id").cast("string"), 256)
).withColumn(
    "email_hash",
    sha2(col("email"), 256)
)

print("=== Hash SHA-256 des Identifiants ===")
df_customers_secure.select(
    "customer_id", "customer_id_hash",
    "email", "email_hash"
).show(5, truncate=False)

print("\n📊 Statistiques Hash :")
print(f"   - Longueur hash SHA-256 : 64 caractères")
print(f"   - Type : Chiffrement unidirectionnel (irreversible)")
print(f"   - Usage : Identification anonymisée pour analyses")

## 3.7  Sécurité et Protection des Données (RGPD)

Pour protéger les données sensibles des clients, nous appliquons deux techniques :
1. **Hash SHA-256** : Chiffrement unidirectionnel des identifiants sensibles (customer_id, email)
2. **Masquage** : Anonymisation partielle des données personnelles (numéro de téléphone)

Ces mesures assurent la conformité RGPD tout en permettant les analyses.

### 4.1 Évolution Mensuelle du CA

In [ ]:
# Préparer les colonnes de date
df_temporal = df_joined.withColumn("year", year("order_date")) \
    .withColumn("month", month("order_date"))

# CA mensuel
df_ca_mensuel = df_temporal.groupBy("year", "month").agg(
    spark_sum("total_amount").alias("ca_mensuel"),
    count("order_id").alias("nb_commandes")
).orderBy("year", "month")

df_ca_mensuel = df_ca_mensuel.withColumn("ca_mensuel", spark_round("ca_mensuel", 2))

print("=== CA Mensuel ===")
df_ca_mensuel.show(24, truncate=False)

In [ ]:
# Window function: CA cumulé et évolution
window_temporal = Window.orderBy("year", "month")

df_ca_mensuel_evolution = df_ca_mensuel.withColumn(
    "ca_cumule",
    spark_sum("ca_mensuel").over(window_temporal)
).withColumn(
    "ca_mois_precedent",
    lag("ca_mensuel", 1).over(window_temporal)
).withColumn(
    "evolution_pct",
    when(
        col("ca_mois_precedent").isNotNull(),
        spark_round(((col("ca_mensuel") - col("ca_mois_precedent")) / col("ca_mois_precedent") * 100), 2)
    )
)

print("=== Évolution Mensuelle du CA ===")
df_ca_mensuel_evolution.show(24, truncate=False)

### 4.2 Ranking des Clients par Période

In [ ]:
# CA par client et par mois
df_client_mensuel = df_temporal.groupBy("year", "month", "customer_id", "first_name", "last_name").agg(
    spark_sum("total_amount").alias("ca_mensuel")
)

# Window pour ranking par mois
window_rank = Window.partitionBy("year", "month").orderBy(col("ca_mensuel").desc())

df_top_clients_mensuel = df_client_mensuel.withColumn(
    "rank",
    rank().over(window_rank)
).filter(col("rank") <= 5)

df_top_clients_mensuel = df_top_clients_mensuel.withColumn("ca_mensuel", spark_round("ca_mensuel", 2))

print("=== TOP 5 Clients par Mois ===")
df_top_clients_mensuel.orderBy("year", "month", "rank").show(30, truncate=False)

### 4.3 Nombre de Commandes par Client avec Numérotation

In [ ]:
# Numéroter les commandes par client (ordre chronologique)
window_row_number = Window.partitionBy("customer_id").orderBy("order_date")

df_orders_numbered = df_joined.withColumn(
    "order_number",
    row_number().over(window_row_number)
)

# Première et dernière commande par client
df_first_last = df_orders_numbered.filter(
    (col("order_number") == 1) | 
    (col("order_number") == df_orders_numbered.groupBy("customer_id").agg(spark_max("order_number")).first()[0])
)

print("=== Premières et Dernières Commandes (échantillon) ===")
df_orders_numbered.select(
    "customer_id", "first_name", "last_name", "order_date", "order_number", "total_amount"
).filter(col("customer_id") < 10).orderBy("customer_id", "order_number").show(20, truncate=False)

## 5. Création de Features - RFM (Recency, Frequency, Monetary)

In [ ]:
# Date de référence (dernière date dans le dataset)
max_date = df_joined.agg(spark_max("order_date")).collect()[0][0]
print(f"Date de référence: {max_date}")

# Calcul RFM
df_rfm = df_joined.groupBy("customer_id", "first_name", "last_name", "customer_segment").agg(
    # Recency: jours depuis la dernière commande
    datediff(to_date(spark.sql(f"SELECT '{max_date}'").first()[0]), spark_max("order_date")).alias("recency"),
    
    # Frequency: nombre de commandes
    count("order_id").alias("frequency"),
    
    # Monetary: montant total dépensé
    spark_sum("total_amount").alias("monetary")
)

df_rfm = df_rfm.withColumn("monetary", spark_round("monetary", 2))

print("=== Analyse RFM ===")
df_rfm.orderBy(col("monetary").desc()).show(20, truncate=False)

In [ ]:
# Segmentation RFM par quartiles
from pyspark.sql.functions import ntile

window_rfm = Window.orderBy(col("recency").asc())  # Plus petit = meilleur
window_fm = Window.orderBy(col("frequency").desc())  # Plus grand = meilleur
window_m = Window.orderBy(col("monetary").desc())  # Plus grand = meilleur

df_rfm_score = df_rfm.withColumn("r_score", ntile(4).over(window_rfm)) \
    .withColumn("f_score", ntile(4).over(window_fm)) \
    .withColumn("m_score", ntile(4).over(window_m))

# Score global RFM
df_rfm_score = df_rfm_score.withColumn(
    "rfm_score",
    col("r_score") + col("f_score") + col("m_score")
)

# Catégorisation
df_rfm_score = df_rfm_score.withColumn(
    "rfm_category",
    when(col("rfm_score") >= 10, "Champion") \
    .when(col("rfm_score") >= 8, "Loyal") \
    .when(col("rfm_score") >= 6, "Potential") \
    .otherwise("At Risk")
)

print("=== Segmentation RFM ===")
df_rfm_score.orderBy(col("rfm_score").desc()).show(20, truncate=False)

In [ ]:
# Distribution par catégorie RFM
print("=== Distribution par Catégorie RFM ===")
df_rfm_score.groupBy("rfm_category").agg(
    count("customer_id").alias("nb_clients"),
    spark_sum("monetary").alias("ca_total"),
    avg("monetary").alias("ca_moyen")
).orderBy(col("ca_total").desc()).show()

## 6. Customer Lifetime Value (CLV)

In [ ]:
# CLV = Valeur moyenne par commande × Fréquence × Durée de vie client
df_clv = df_joined.groupBy("customer_id", "first_name", "last_name", "customer_segment").agg(
    count("order_id").alias("nb_commandes"),
    spark_sum("total_amount").alias("ca_total"),
    avg("total_amount").alias("ca_moyen_commande"),
    spark_min("order_date").alias("premiere_commande"),
    spark_max("order_date").alias("derniere_commande")
)

# Durée de vie (en jours)
df_clv = df_clv.withColumn(
    "duree_vie_jours",
    datediff(col("derniere_commande"), col("premiere_commande"))
)

# Fréquence d'achat (commandes par jour)
df_clv = df_clv.withColumn(
    "frequence_achat",
    when(
        col("duree_vie_jours") > 0,
        spark_round(col("nb_commandes") / col("duree_vie_jours"), 4)
    ).otherwise(0)
)

# CLV estimé (simplifié)
df_clv = df_clv.withColumn(
    "clv_estime",
    spark_round(col("ca_moyen_commande") * col("nb_commandes") * 2, 2)  # × 2 pour projection future
)

df_clv = df_clv.withColumn("ca_total", spark_round("ca_total", 2)) \
    .withColumn("ca_moyen_commande", spark_round("ca_moyen_commande", 2))

print("=== Customer Lifetime Value ===")
df_clv.orderBy(col("clv_estime").desc()).show(20, truncate=False)

In [ ]:
# Statistiques CLV par segment
print("=== CLV par Segment ===")
df_clv.groupBy("customer_segment").agg(
    count("customer_id").alias("nb_clients"),
    avg("clv_estime").alias("clv_moyen"),
    spark_max("clv_estime").alias("clv_max"),
    avg("frequence_achat").alias("freq_achat_moyen")
).orderBy(col("clv_moyen").desc()).show()

## 7. Sauvegarde des Résultats

In [ ]:
# Sauvegarder les datasets transformés
output_path = "../data/processed/"

# 1. Dataset jointure complète
df_joined.write.mode("overwrite").parquet(f"{output_path}customers_orders_joined.parquet")
print(" customers_orders_joined.parquet sauvegardé")

# 2. CA par client
df_ca_client.write.mode("overwrite").parquet(f"{output_path}ca_par_client.parquet")
print(" ca_par_client.parquet sauvegardé")

# 3. CA mensuel avec évolution
df_ca_mensuel_evolution.write.mode("overwrite").parquet(f"{output_path}ca_mensuel_evolution.parquet")
print(" ca_mensuel_evolution.parquet sauvegardé")

# 4. Analyse RFM
df_rfm_score.write.mode("overwrite").parquet(f"{output_path}rfm_analysis.parquet")
print(" rfm_analysis.parquet sauvegardé")

# 5. Customer Lifetime Value
df_clv.write.mode("overwrite").parquet(f"{output_path}customer_lifetime_value.parquet")
print(" customer_lifetime_value.parquet sauvegardé")

# 6. CA par catégorie
df_ca_categorie.write.mode("overwrite").parquet(f"{output_path}ca_par_categorie.parquet")
print(" ca_par_categorie.parquet sauvegardé")

# 7. CA par pays
df_ca_pays.write.mode("overwrite").parquet(f"{output_path}ca_par_pays.parquet")
print(" ca_par_pays.parquet sauvegardé")

print("\n=== Tous les fichiers ont été sauvegardés avec succès ! ===")

## 8. Résumé des Transformations Effectuées

### Jointures
-  Jointure INNER entre customers et orders sur `customer_id`
-  Préservation de l'intégrité référentielle

### Agrégations
-  CA par client (total, moyen, min, max)
-  CA par pays
-  CA par catégorie de produit
-  CA par segment client
-  CA mensuel avec évolution

### Window Functions
-  CA cumulé mensuel
-  Évolution par rapport au mois précédent
-  Ranking des clients par période
-  Numérotation des commandes par client

### Features Avancées
-  Analyse RFM (Recency, Frequency, Monetary)
-  Segmentation RFM (Champion, Loyal, Potential, At Risk)
-  Customer Lifetime Value (CLV)
-  Fréquence d'achat

### Fichiers Générés
Tous les résultats sont sauvegardés en format Parquet dans `data/processed/` pour utilisation dans la Part 3.

In [ ]:
# Fermeture de la session Spark
# spark.stop()
print("Session Spark active - Prête pour Part 3")